In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
from pytorch_transformers import modeling_bert
import numpy as np
import torch
from pytorch_transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from tqdm import tqdm
import pdb
import torch.nn.functional as F
from torch import optim
import sys
model_name = 'Original_Bert'
import time
import utils
import imp
#from utils import batch_sent_loader
import os
from sklearn.metrics import confusion_matrix
import json

In [14]:
#Set up parameters
imp.reload(utils)
hpara1 = utils.hpara()
hpara1.ratio = 2
hpara1.batch_size = 8
hpara1.word_layers=1
hpara1.sent_layers=1
hpara1.use_position_embedding = False
hpara1.use_angular = False
hpara1.word_lr = 5e-05
hpara1.sent_lr = 0.0002
hpara1.accumulation_steps = 40

In [15]:
hpara1.use_PU_Bert = False

In [16]:
if hpara1.use_PU_Bert:
    pretrain_model_dir = '/gpfs/qlong/home/tzzhang/nlp_test/bert/mimic_based_complete_model'
else:
    pretrain_model_dir = '/gpfs/qlong/home/tzzhang/bioBert/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000'
vocab_file = os.path.join(pretrain_model_dir,'vocab.txt')
bert_config_file = os.path.join(pretrain_model_dir,'bert_config.json')
      

In [17]:
imp.reload(utils)
training_generator,validation_generator,dummy_generator = utils.load_data(ratio=hpara1.ratio,clarity=True)
tokenizer = utils._load_tf_tokenizer(vocab_file = vocab_file)

In [18]:
# Set up the word embedding model but do not load the pre-trained parameters
config = modeling_bert.BertConfig.from_json_file(bert_config_file)
config.num_hidden_layers = hpara1.word_layers
config.output_attentions = True
model_word = modeling_bert.BertModel(config)

In [19]:
para = model_word.state_dict()
att_layer_weight = para['encoder.layer.0.attention.self.query.weight']
print(att_layer_weight[0,:10])

tensor([ 0.0031, -0.0138,  0.0488, -0.0027,  0.0052, -0.0378, -0.0397, -0.0166,
        -0.0285,  0.0447])


In [20]:
import re
# Set up the word embedding model and load the pre-trained parameters
if hpara1.use_PU_Bert:
    model_state_dict = './PU_pretrained_model/save_word_complete0.bin'
else:
    model_state_dict = pretrain_model_dir + '/pytorch_model.bin'
pretrained_dict = torch.load(model_state_dict)
model_dict = model_word.state_dict()
matched_dict = {}
for k in pretrained_dict.keys():
    try:
        new_k = re.search(r'(bert\.)(.*)',k).group(2)
    except:
        continue
    if new_k in model_dict:
        matched_dict[new_k] = pretrained_dict[k]
model_dict.update(matched_dict)
model_word.load_state_dict(model_dict)

<All keys matched successfully>

In [21]:
config_doc = modeling_bert.BertConfig.from_json_file(bert_config_file)
config_doc.num_hidden_layers = hpara1.sent_layers
config_doc.output_attentions = True
use_advanced_loss = hpara1.use_angular
use_position_embedding = hpara1.use_position_embedding
model_sent = modeling_bert.BertModel_no_embedding(config_doc,use_advanced_loss=use_advanced_loss,use_position_embedding=use_position_embedding)

In [22]:
para = model_word.state_dict()
att_layer_weight = para['encoder.layer.0.attention.self.query.weight']
print(att_layer_weight[0,:10])

tensor([-0.0465,  0.0266, -0.0032,  0.0086,  0.0237,  0.0398, -0.0429,  0.0951,
         0.0558,  0.0189])


In [23]:
model_word = model_word.cuda()
model_sent = model_sent.cuda()

In [24]:
date = str(list(time.localtime())[0:3]).replace(', ','_')
save_dir = './exp/'+model_name+'_'+date[1:-1]+'_1layer/'

if not os.path.exists(save_dir):
    cmd = 'mkdir -p ' + save_dir
    os.system(cmd)
     
with open(save_dir + 'hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)
    
do_train = True
do_test = True

max_epoch = hpara1.max_epoch
log = 'Iter {}/{}, Loss={:.3f},accu={:.3f},time={:.3f}\n'
from tqdm import tqdm
batch_size = hpara1.batch_size
accumulation_steps = hpara1.accumulation_steps
max_sent_len = hpara1.max_sent_len
max_doc_len = hpara1.max_doc_len
#progress_bar = tqdm(enumerate(training_generator))
para_dict = {}
hpara_list = []

In [ ]:
utils.model_train_and_test(hpara1,model_word,model_sent,save_dir,training_generator,validation_generator,tokenizer=tokenizer)

0
1000
2000
3000
4000
Iter 0/60, Loss=3203.680,accu=0.652,time=854.353

Test Iter 0/60, Loss=768.333,accu=0.667,auc=0.770,time=132.597

[[789   0]
 [394   0]]
0
1000
2000
3000
4000
Iter 1/60, Loss=3047.954,accu=0.667,time=865.277

Test Iter 1/60, Loss=751.259,accu=0.667,auc=0.749,time=130.537

[[789   0]
 [394   0]]
0
1000
2000
3000
4000
Iter 2/60, Loss=2734.345,accu=0.673,time=860.616

Test Iter 2/60, Loss=598.831,accu=0.706,auc=0.783,time=133.848

[[663 126]
 [222 172]]
0
1000
2000
3000
4000
Iter 3/60, Loss=2363.025,accu=0.712,time=859.794

Test Iter 3/60, Loss=554.649,accu=0.721,auc=0.841,time=130.364

[[745  44]
 [286 108]]
0
1000
2000
3000
4000
Iter 4/60, Loss=2129.762,accu=0.760,time=851.371

Test Iter 4/60, Loss=530.027,accu=0.771,auc=0.866,time=133.960

[[720  69]
 [202 192]]
0
1000
2000
3000
4000
Iter 5/60, Loss=2053.186,accu=0.780,time=849.943

Test Iter 5/60, Loss=478.229,accu=0.806,auc=0.880,time=131.520

[[686 103]
 [126 268]]
0
1000
2000
3000
4000
Iter 6/60, Loss=1916.173